In [1]:
import yaml
from collections import defaultdict
from slpp import slpp as lua #pip install git+https://github.com/SirAnthony/slpp
import pandas as pd

In [2]:
path_live = "/Applications/World of Warcraft/_classic_/WTF/Account/BLUEM/SavedVariables/ArkInventory.lua" 
path_local = "data/ArkInventory.lua"

In [3]:
try:
    with open(path_live, 'r') as f:
        data = lua.decode('{'+f.read()+'}')
except:        
    with open(path_local, 'r') as f:
        data = lua.decode('{'+f.read()+'}')

characters = data['ARKINVDB']['global']['player']['data']        

In [4]:
# Search through inventory data to create dictionary of all items and counts, also counts total monies
monies = {}
items = defaultdict(int)

for ckey in characters:
    character = characters[ckey]
    monies[ckey] = int(character.get('info').get('money', 0))
    
    # Get Bank, Inventory, Character, Mailbox etc
    location_slots = character.get('location', [])
    for lkey in location_slots:
        location_slot = location_slots[lkey]
        if location_slot:
            bag_slots = location_slot['bag']
            
            # Get the items from each of the bags, add to master list
            for bag in bag_slots:
                for item in bag.get('slot', []):                       
                    if item.get('h') and item.get('count'):
                        item_desc = item.get('h').split('[')[1].split(']')[0]
                        
                        items[item_desc] += item.get('count')
    
total_monies = sum(list(monies.values()))/10000
print(f"Total Monies: {total_monies:.2f}")

Total Monies: 220.95


In [5]:
# Converts to pandas
df_raw = pd.DataFrame.from_dict(items, orient='index')
df_raw.columns = ['count']

In [10]:
# Pulls our list of items of interest, and filters dataframe accordingly
path_items_of_interest = 'items_of_interest.yaml'

with open(path_items_of_interest, 'r') as f:
    items_of_interest = yaml.load(f, Loader=yaml.FullLoader)
    
for key in items_of_interest:
    for item in items_of_interest[key]:
        if item not in df_raw.index:
            df_raw.loc[item, 'count'] = 0
    
    df_raw.loc[items_of_interest[key], 'type'] = key
    
df_raw['count'] = df_raw['count'].astype(int)

df = df_raw.dropna().sort_values(['type','count'], ascending=[False, False])

In [11]:
df

,count,type
Plaguebloom,171,Snatch
Gromsblood,89,Snatch
Goldthorn,86,Snatch
Sungrass,86,Snatch
Mountain Silversage,82,Snatch
Small Flame Sac,31,Snatch
Ghost Mushroom,27,Snatch
Oily Blackmouth,21,Snatch
Wild Steelbloom,2,Snatch
Stonescale Eel,0,Snatch


In [8]:
# Minus out known amounts per person